In [16]:
# Import libraries
import betfairlightweight
from betfairlightweight import filters
import pandas as pd
import numpy as np
import os
import datetime
import json

# Change this certs path to wherever you're storing your certificates
certs_path = "/home/naing/certs/"

# Change these login details to your own
my_username = "mightytranks@gmail.com"
my_password = "OsGiwuevzRf4925Lfty!"
my_app_key = "X9EGwt5zbpg8o5xk"

trading = betfairlightweight.APIClient(username=my_username,
                                       password=my_password,
                                       app_key=my_app_key,
                                       certs=certs_path)

trading.login()


myRaceID = '1.199996222'

In [17]:
def process_runner_books(runner_books):
    '''
    This function processes the runner books and returns a DataFrame with the best back/lay prices + vol for each runner
    :param runner_books:
    :return:
    '''
    best_back_prices = [runner_book.ex.available_to_back[0].price
                        if runner_book.ex.available_to_back
                        else 1.01
                        for runner_book
                        in runner_books]
    best_back_sizes = [runner_book.ex.available_to_back[0].size
                       if runner_book.ex.available_to_back
                       else 1.01
                       for runner_book
                       in runner_books]

    best_lay_prices = [runner_book.ex.available_to_lay[0].price
                       if runner_book.ex.available_to_lay
                       else 1000.0
                       for runner_book
                       in runner_books]
    best_lay_sizes = [runner_book.ex.available_to_lay[0].size
                      if runner_book.ex.available_to_lay
                      else 1.01
                      for runner_book
                      in runner_books]
    
    selection_ids = [runner_book.selection_id for runner_book in runner_books]
    last_prices_traded = [runner_book.last_price_traded for runner_book in runner_books]
    total_matched = [runner_book.total_matched for runner_book in runner_books]
    statuses = [runner_book.status for runner_book in runner_books]
    scratching_datetimes = [runner_book.removal_date for runner_book in runner_books]
    adjustment_factors = [runner_book.adjustment_factor for runner_book in runner_books]

    df = pd.DataFrame({
        'Selection ID': selection_ids,
        'Best Back Price': best_back_prices,
        'Best Back Size': best_back_sizes,
        'Best Lay Price': best_lay_prices,
        'Best Lay Size': best_lay_sizes,
        'Last Price Traded': last_prices_traded,
        'Total Matched': total_matched,
        'Status': statuses,
        'Removal Date': scratching_datetimes,
        'Adjustment Factor': adjustment_factors
    })
    return df

In [18]:
# Create a price filter. Get all traded and offer data
price_filter = filters.price_projection(
    price_data=['EX_BEST_OFFERS']
)

# Request market books
market_books = trading.betting.list_market_book(
    market_ids=[myRaceID],
    price_projection=price_filter
)

# Grab the first market book from the returned list as we only requested one market 
market_book = market_books[0]

runners_df = process_runner_books(market_book.runners)

runners_df

,Selection ID,Best Back Price,Best Back Size,Best Lay Price,Best Lay Size,Last Price Traded,Total Matched,Status,Removal Date,Adjustment Factor
0,41419585,44.0,25.64,140.0,19.14,None,0.0,ACTIVE,None,None
1,41419586,34.0,28.40,110.0,17.58,None,0.0,ACTIVE,None,None
2,41419587,32.0,25.05,90.0,18.39,None,0.0,ACTIVE,None,None
3,41419588,80.0,40.64,270.0,28.11,None,0.0,ACTIVE,None,None
4,41419589,95.0,26.98,400.0,26.94,None,0.0,ACTIVE,None,None
5,41419592,38.0,32.07,100.0,24.52,None,0.0,ACTIVE,None,None
6,41419593,14.5,82.91,25.0,47.75,None,0.0,ACTIVE,None,None
7,41419594,13.5,99.82,23.0,47.53,None,0.0,ACTIVE,None,None
8,41419595,30.0,39.54,55.0,19.36,None,0.0,ACTIVE,None,None
9,41419596,38.0,28.27,85.0,21.14,None,0.0,ACTIVE,None,None


In [23]:
fav_selection_id = '41419608'
fav_price = 5 # choose the lowest

limit_order_filter = filters.limit_order(
    price=fav_price,
    persistence_type='LAPSE',
    bet_target_type='PAYOUT',
    bet_target_size=50 # use this if i want $1500 as liability
)

instructions_filter = filters.place_instruction(
    selection_id = str(fav_selection_id),
    order_type = "LIMIT",
    side="LAY",
    limit_order=limit_order_filter
)

instructions_filter

{'orderType': 'LIMIT',
 'selectionId': '41419608',
 'side': 'LAY',
 'limitOrder': {'price': 5,
  'persistenceType': 'LAPSE',
  'betTargetType': 'PAYOUT',
  'betTargetSize': 50}}

In [24]:
order = trading.betting.place_orders(
    market_id = myRaceID,
    customer_strategy_ref='Naing_maker',
    instructions=[instructions_filter]
)

In [13]:
# listClearedOrders
cleared_orders = trading.betting.list_cleared_orders(bet_status="SETTLED",
                                                    market_ids=[myRaceID])


# Create a DataFrame from the orders
pd.DataFrame(cleared_orders._data['clearedOrders'])

,eventTypeId,eventId,marketId,selectionId,handicap,betId,placedDate,persistenceType,orderType,side,betOutcome,priceRequested,settledDate,lastMatchedDate,betCount,priceMatched,priceReduced,sizeSettled,profit,customerStrategyRef
0,4339,31512294,1.199996463,41419623,0.0,270817741934,2022-06-08T13:12:12.000Z,LAPSE,LIMIT,LAY,WON,14.0,2022-06-08T13:17:22.000Z,2022-06-08T13:12:12.000Z,1,14.0,False,5.0,5.0,Naing_maker
1,4339,31512294,1.199996463,41419623,0.0,270817786787,2022-06-08T13:12:41.000Z,LAPSE,LIMIT,LAY,WON,14.0,2022-06-08T13:17:22.000Z,2022-06-08T13:12:42.000Z,1,14.0,False,100.0,100.0,Naing_maker


In [15]:
# Get the profit/loss - this returns a list
pl = trading.betting.list_market_profit_and_loss(market_ids=[myRaceID], 
                                                 include_bsp_bets='true', 
                                                 include_settled_bets='true')
# Create a profit/loss DataFrame
pl_df = pd.DataFrame(pl[0]._data['profitAndLosses']).assign(marketId=pl[0].market_id)
pl_df

IndexError: list index out of range